# Analyze text sentiment:The machine learning approach

This project is based on Andrew Trask 
[Sentiment project](https://github.com/udacity/deep-learning/tree/master/sentiment-network).

The dataset is part of the [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/) publication.

In [1]:
from collections import Counter
import os
import math
from random import randint
import sys
import time
from IPython.display import Image


import numpy as np

from lib.reviews.load_reviews import load_reviews
from lib.activation_functions.sigmoid import sigmoid
from lib.derivatives.sigmoid_derivative import sigmoid_derivative

### Load the reviews and labels data

In [2]:
POSITIVE_DATASET_PATH = "dataset/positive_reviews.txt"
positive_reviews = load_reviews(POSITIVE_DATASET_PATH)

positive_reviews[0]

'I find it remarkable that so little was actually done with the story of the abomb and its development for decades after the Manhattan Project was completed My suspicion is that this was due to serious fears in the movie and entertainment industries in the s through the s with McCarthyism and related national security phobias including the Hollywood blacklist There was one film in the s with Robert Taylor about Col Paul Tibbits who flew the Enola Gay in the Hiroshima bombing but otherwise nothing else One could glance at a side issue tragedy the sinking of the USS Indianapolis soon after the delivery of the bombs to Tinian in Robert Shaws description of the shark attacks on the survivors in JAWS But the actual trials and tribulations of Groves Oppenheimer and their team was not considered filmablebr br And then in  two films appeared I have reviewed one already DAY ONE which I feel is the better of the two in discussing the lengthy technical and emotional and political problems in the 

In [3]:
NEGATIVE_DATASET_PATH = "dataset/negative_reviews.txt"
negative_reviews = load_reviews(NEGATIVE_DATASET_PATH)

negative_reviews[0]

'Boring movie Poor plot Poor actors The movie happens in a room supposed to be in Morocco but actually in some American city The Arab terrorists are the patriots the blonde patriot is the Arab terroristDAMNbr br There is something good about this movie though thats why the score is  out of  The director turns the ridiculous stereotype about terrorism the media feeds us every day into the real thing the terrorists are Americans or western people if you likebr br The movie is divided into two parts The first part of the movie concerns the Dutchman travel  seconds while the second part is about the staying in the amazing dark brown room  hour and somethingbr br The Dutch guy is going to deliver money in Morocco to some charity organization gets off the plain takes the bus and ends up kidnapped in a dark brown room He is kidnapped with another guy that is shot after telling They will not shoot at us The Dutch survivor is forced to play chess with a Morpheuslike Arab guy for so long that yo

### Create the words counters

We'll create three `Counter` objects, one for words from postive reviews, one for words from negative reviews, and one for all the words.

In [4]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [5]:
def get_words_count(reviews):
    words_counts = Counter()
    
    for index in range(len(reviews)):
        words = reviews[index].split(' ')
        
        for word in words:
            words_counts[word] += 1
            
    return words_counts

positive_counts = get_words_count(positive_reviews)
negative_counts = get_words_count(negative_reviews)

total_counts = positive_counts + negative_counts

Examine the most common words in positive reviews

In [6]:
positive_counts.most_common()

[('the', 144410),
 ('and', 82816),
 ('a', 76738),
 ('of', 74140),
 ('to', 63529),
 ('is', 53902),
 ('in', 45347),
 ('I', 33703),
 ('that', 31574),
 ('it', 31043),
 ('br', 27999),
 ('this', 26696),
 ('as', 22970),
 ('with', 21696),
 ('The', 21696),
 ('', 21219),
 ('was', 21213),
 ('for', 20539),
 ('film', 19232),
 ('movie', 17308),
 ('but', 16154),
 ('on', 15865),
 ('his', 15596),
 ('are', 14242),
 ('you', 13167),
 ('not', 12817),
 ('be', 11861),
 ('have', 11859),
 ('one', 11631),
 ('by', 11367),
 ('he', 11196),
 ('an', 10725),
 ('at', 10378),
 ('who', 10294),
 ('all', 9683),
 ('from', 9634),
 ('its', 9154),
 ('has', 8715),
 ('her', 8475),
 ('like', 8010),
 ('about', 7903),
 ('so', 7509),
 ('out', 7485),
 ('they', 7474),
 ('very', 7462),
 ('This', 7388),
 ('or', 7275),
 ('more', 6958),
 ('good', 6651),
 ('just', 6391),
 ('some', 6359),
 ('It', 6144),
 ('what', 5955),
 ('their', 5926),
 ('great', 5923),
 ('when', 5847),
 ('see', 5774),
 ('story', 5707),
 ('which', 5577),
 ('time', 5570),

And the respective most common words in negative reviews

In [7]:
negative_counts.most_common()

[('the', 139017),
 ('a', 75880),
 ('and', 68762),
 ('of', 67174),
 ('to', 66524),
 ('is', 48337),
 ('in', 40437),
 ('I', 37952),
 ('that', 33232),
 ('this', 32683),
 ('it', 31994),
 ('br', 29148),
 ('was', 25662),
 ('', 25219),
 ('movie', 23157),
 ('The', 22535),
 ('with', 20073),
 ('for', 19983),
 ('as', 18377),
 ('but', 16949),
 ('film', 16785),
 ('on', 16097),
 ('have', 15290),
 ('are', 14314),
 ('not', 14212),
 ('be', 13966),
 ('you', 13785),
 ('one', 11217),
 ('at', 11151),
 ('his', 11078),
 ('like', 10583),
 ('all', 10196),
 ('they', 10037),
 ('an', 9846),
 ('just', 9749),
 ('by', 9746),
 ('he', 9507),
 ('or', 9446),
 ('from', 9353),
 ('so', 9314),
 ('who', 8854),
 ('about', 8531),
 ('out', 8509),
 ('its', 8392),
 ('some', 7673),
 ('has', 7363),
 ('This', 7034),
 ('her', 6994),
 ('good', 6727),
 ('would', 6725),
 ('even', 6619),
 ('bad', 6550),
 ('if', 6519),
 ('no', 6473),
 ('more', 6266),
 ('up', 6249),
 ('only', 6238),
 ('what', 6096),
 ('were', 5989),
 ('really', 5804),
 ('th

As you can see, common words like "the" appear very often in both positive and negative reviews. Instead of finding the most common words in positive or negative reviews, what you really want are the words found in positive reviews more often than in negative reviews, and vice versa. To accomplish this, you'll need to calculate the ratios of word usage between positive and negative reviews.

In [8]:
pos_neg_ratios = Counter()

for word in positive_counts:
    if(positive_counts[word] > 100 or negative_counts[word] > 100):
        pos_neg_ratios[word] = math.log(positive_counts[word] / (negative_counts[word] + 1))

Examine the calculated ratios for a few words:

In [9]:
print(positive_counts["the"])
print(negative_counts["the"])
print(pos_neg_ratios["the"])

print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

144410
139017
0.038053054988839714
Pos-to-neg ratio for 'the' = 0.038053054988839714
Pos-to-neg ratio for 'amazing' = 1.3419941022233106
Pos-to-neg ratio for 'terrible' = -2.074219597698684


Neutral word have a ratio value close to 0. Words expected to see more often in positive reviews – like "amazing" – have a ratio greater than 0. Words with a ratio lower than 0 were expected to be more often in negative reviews.
Extremely positive and extremely negative words will have positive-to-negative ratios with similar magnitudes but opposite signs.

### Build the neural network

Assign a seed to our random number generator to ensure we get reproducable results during development.

In [10]:
np.random.seed(1)

Define a network learning rate.

In [11]:
learning_rate = 0.01

Define a polarity cutoff to exclude values very close to 0.

In [12]:
POLARITY_CUTOFF = 0.02

Create the words indexes dictionary processing the positive and negative reviews and keeping only the words with a ratio greater than the polarity cutoff.

In [13]:
word_index = 0
words_indexes_dictionary = {}

for word in pos_neg_ratios:
    if(abs(pos_neg_ratios[word]) > POLARITY_CUTOFF):
        words_indexes_dictionary[word] = word_index
        word_index += 1

Define the data sets for training and testing the neural network.

In [50]:
NEGATIVE = 0
POSITIVE = 1

reviews = []
labels = []

# Insert positive reviews

reviews = positive_reviews[:]
labels = [POSITIVE] * len(reviews)

# Insert randomly negative reviews

for review_index in range(len(negative_reviews)):
    index = randint(0, len(reviews))
    reviews.insert(index, negative_reviews[review_index])
    labels.insert(index, NEGATIVE)

train_reviews = reviews[:22000]
test_reviews = reviews[-3000:]

train_labels = labels[:22000]
test_labels = labels[-3000:]

Build the neural network structure having only an hidden layer.

In [51]:
INPUT_LAYER_NODES = len(words_indexes_dictionary)
HIDDEN_LAYER_NODES = 10
OUTPUT_LAYER_NODES = 1

input_to_hidden_weights = np.zeros((INPUT_LAYER_NODES, HIDDEN_LAYER_NODES))
hidden_to_output_weights = np.random.normal(0.0, HIDDEN_LAYER_NODES ** -0.5, 
                                            (HIDDEN_LAYER_NODES, OUTPUT_LAYER_NODES))

hidden_layer = np.zeros((1, HIDDEN_LAYER_NODES))

### Train the neural network

Loop through all the given reviews and run a forward and backward pass, updating weights for every item.

In [52]:
correct_predictions = 0
 
for review_index in range(len(train_reviews)):
    review = train_reviews[review_index]
    label = train_labels[review_index]
    
    words = review.split(' ')
    
    # Prepare the list of unique word indexes found on current review
    
    words_indexes = set()
    
    for word in words:
        if word in words_indexes_dictionary:
            word_index = words_indexes_dictionary[word]
            words_indexes.add(word_index)
            
    ## The forward pass through the network
            
    # Calculate the hidden layer values with the input to hidden weights
            
    hidden_layer = np.zeros((1, HIDDEN_LAYER_NODES))
    
    for word_index in words_indexes:
        hidden_layer += input_to_hidden_weights[word_index]
        
    # Calculate the output value multiplying the hidden layer values by the hidden to output weights
        
    output = hidden_layer.dot(hidden_to_output_weights)
    output = sigmoid(output)
    
    ## The back propagation pass
    
    # Calculate the output error and delta
    
    error = output - label
    output_delta = error * sigmoid_derivative(output)
    
    # Calculate the hidden error and delta
    
    hidden_errors = output_delta.dot(hidden_to_output_weights.T)
    hidden_deltas = hidden_errors
    
    # Update the network weights using the calculated deltas
    
    hidden_to_output_weights -= hidden_layer.T.dot(output_delta) * learning_rate
    
    for word_index in words_indexes:
        input_to_hidden_weights[word_index] -= hidden_deltas[0] * learning_rate
        
    # Keep track of correct predictions
    
    if(np.abs(error) < 0.5):
        correct_predictions += 1
        
    sys.stdout.write("\rCorrect predictions: " + str(correct_predictions) + 
                     " - Trained: " + str(review_index) +
                     " - Testing Accuracy:" + str(correct_predictions * 100 / float(review_index + 1))[:4] + "%")

Correct predictions: 199 - Trained: 358 - Testing Accuracy:55.4%

Correct predictions: 405 - Trained: 693 - Testing Accuracy:58.3%

Correct predictions: 674 - Trained: 1066 - Testing Accuracy:63.1%

Correct predictions: 960 - Trained: 1432 - Testing Accuracy:66.9%

Correct predictions: 1213 - Trained: 1761 - Testing Accuracy:68.8%

...

Correct predictions: 17078 - Trained: 20660 - Testing Accuracy:82.6%

Correct predictions: 17365 - Trained: 20991 - Testing Accuracy:82.7%

Correct predictions: 17646 - Trained: 21315 - Testing Accuracy:82.7%

Correct predictions: 17976 - Trained: 21686 - Testing Accuracy:82.8%

Correct predictions: 18255 - Trained: 21999 - Testing Accuracy:82.9%

### Test the neural network

Use the test_labels to calculate the accuracy of previous predictions

In [53]:
correct_predictions = 0

for review_index in range(len(test_reviews)):
    review = test_reviews[review_index]
    label = test_labels[review_index]
    
    # Prepare the list of unique word indexes found on current review
    
    review_words = set()
    
    for word in review.split(' '):
        review_words.add(word)
        
    words_indexes = []
        
    for review_word in review_words:
        if(review_word in words_indexes_dictionary):
            words_indexes.append(words_indexes_dictionary[review_word])
            
    ## The forward pass through the network
            
    # Calculate the hidden layer values with the input to hidden weights
        
    hidden_layer = np.zeros((OUTPUT_LAYER_NODES, HIDDEN_LAYER_NODES))
    
    for word_index in words_indexes:
        hidden_layer += input_to_hidden_weights[word_index]
    
    # Calculate the output value multiplying the hidden layer values by the hidden to output weights
    
    output = hidden_layer.dot(hidden_to_output_weights)
    output = sigmoid(output)
    
    error = output - label
    
    # Keep track of correct predictions
    
    if(np.abs(error) < 0.5):
        correct_predictions += 1
     
    sys.stdout.write("\rCorrect predictions: " + str(correct_predictions) \
                     + " - Trained: " + str(review_index) \
                     + " - Testing Accuracy:" \
                     + str(correct_predictions * 100 / float(review_index + 1))[:4] + "%")

Correct predictions: 572 - Trained: 678 - Testing Accuracy:84.2%

Correct predictions: 1061 - Trained: 1253 - Testing Accuracy:84.6%

Correct predictions: 1944 - Trained: 2278 - Testing Accuracy:85.3%

Correct predictions: 2557 - Trained: 2999 - Testing Accuracy:85.2%